In [1]:
pathBase =  '..'

Carga de Bibliotecas

In [2]:
# librerias basicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



# Carga de archivos #

**Datos Seul**

In [3]:
path =  pathBase + '/data/raw/public_bikes_2018/seul/SeoulBikeData.csv'
dataRawSeul = pd.read_csv( path  , encoding='ISO-8859-1' , engine='python')

#agrega columna solo año
dataRawSeul['anho'] = pd.to_datetime(  dataRawSeul['Date'] , format='%d/%m/%Y'  ).dt.year

# se deja la data 2018
dataSeul = dataRawSeul[ dataRawSeul['anho']==2018 ] 


#dataSeul=dataSeul.drop(['anho'],axis=1)

dataSeul.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,anho
744,01/01/2018,206,0,-3.2,40,0.5,1358,-14.9,0.0,0.0,0.0,Winter,Holiday,Yes,2018
745,01/01/2018,230,1,-3.3,41,0.7,2000,-14.7,0.0,0.0,0.0,Winter,Holiday,Yes,2018
746,01/01/2018,178,2,-3.7,42,0.9,2000,-14.7,0.0,0.0,0.0,Winter,Holiday,Yes,2018
747,01/01/2018,119,3,-4.0,44,1.0,2000,-14.4,0.0,0.0,0.0,Winter,Holiday,Yes,2018
748,01/01/2018,62,4,-4.2,53,1.1,1994,-12.3,0.0,0.0,0.0,Winter,Holiday,Yes,2018


**Datos Toronto**

Datos del clima de toronto


https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=48549&Year=2018&Month=2&timeframe=1&submit=Download+Data






In [4]:
from datetime import date, datetime

def get_season(fila):
  # funciomn para determinar le estacion de toronto
  fecha =   datetime.strptime(fila.Date, '%Y-%m-%d').date()

  if (  date(2018,  1,  1) <= fecha <= date(2018,  3, 20) ) :
    estacion = 'Winter'  
  elif ( date(2018,  3, 21) <= fecha <=  date(2018,  6, 20) ):
    estacion = 'Spring'
  elif ( date(2018,  6, 21) <= fecha <=   date(2018,  9, 22)   ):
    estacion = 'Summer'
  elif (  date(2018,  9, 23) <= fecha <=  date(2018, 12, 20)  ):
    estacion = 'Autumn'
  elif ( date(2018, 12, 21) <= fecha <=  date(2018, 12, 31) ) :
    estacion = 'Winter'

  return(estacion)




pathQ1 = pathBase + '/data/raw/public_bikes_2018/toronto/Bike Share Toronto Ridership_Q1 2018.csv'
pathQ2 = pathBase + '/data/raw/public_bikes_2018/toronto/Bike Share Toronto Ridership_Q2 2018.csv'
pathQ3 = pathBase + '/data/raw/public_bikes_2018/toronto/Bike Share Toronto Ridership_Q3 2018.csv'
pathQ4 = pathBase + '/data/raw/public_bikes_2018/toronto/Bike Share Toronto Ridership_Q4 2018.csv'

dataRawQ1 = pd.read_csv( pathQ1  , encoding='ISO-8859-1' , engine='python')
dataRawQ2 = pd.read_csv( pathQ2  , encoding='ISO-8859-1' , engine='python')
dataRawQ3 = pd.read_csv( pathQ3  , encoding='ISO-8859-1' , engine='python')
dataRawQ4 = pd.read_csv( pathQ4  , encoding='ISO-8859-1' , engine='python')

#Se unen los registros de 2018
dataRawToronto = pd.concat([dataRawQ1, dataRawQ2 , dataRawQ3, dataRawQ4 ] , axis=0)


# se da formato a los atributos de fecha y hora
dataRawToronto['Date'] = pd.to_datetime(  dataRawToronto['trip_start_time'] , format='%m/%d/%Y %H:%M'  ).dt.strftime('%Y-%m-%d') 
dataRawToronto['Hour'] = pd.to_datetime(  dataRawToronto['trip_start_time'] , format='%m/%d/%Y %H:%M'  ).dt.hour


# como la data de Toronto esta detallada por viaje, se agrupan los datos por fecha y hora y se cuentan, esto para asemejar a la estructura de data de Seul
dataTorontoGrp = dataRawToronto[['Date' , 'Hour']].groupby(['Date','Hour']).size().reset_index(name='counts')    
dataTorontoGrp.columns = dataTorontoGrp.columns.str.replace('counts', 'Rented Bike Count')


 
dataTorontoGrp['Seasons'] = dataTorontoGrp.apply(get_season, axis=1)


dataTorontoGrp.head()

,Date,Hour,Rented Bike Count,Seasons
0,2018-01-01,0,4,Winter
1,2018-01-01,1,10,Winter
2,2018-01-01,2,6,Winter
3,2018-01-01,3,14,Winter
4,2018-01-01,4,6,Winter


Ahora, con la data agrupada, se agregan los datos del clima de Toronto

In [5]:
# Se cargan los datos de l clima para Toronto 2018

pathC01 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_01-2018_P1H.csv'
pathC02 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_02-2018_P1H.csv'
pathC03 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_03-2018_P1H.csv'
pathC04 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_04-2018_P1H.csv'
pathC05 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_05-2018_P1H.csv'
pathC06 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_06-2018_P1H.csv'
pathC07 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_07-2018_P1H.csv'
pathC08 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_08-2018_P1H.csv'
pathC09 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_09-2018_P1H.csv'
pathC10 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_10-2018_P1H.csv'
pathC11 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_11-2018_P1H.csv'
pathC12 = pathBase + '/data/raw/datasetsTorontoClima/en_climate_hourly_ON_6158359_12-2018_P1H.csv'



dataRawC01 = pd.read_csv( pathC01  , encoding='utf8' , engine='python')
dataRawC02 = pd.read_csv( pathC02  , encoding='utf8' , engine='python')
dataRawC03 = pd.read_csv( pathC03  , encoding='utf8' , engine='python')
dataRawC04 = pd.read_csv( pathC04  , encoding='utf8' , engine='python')
dataRawC05 = pd.read_csv( pathC05  , encoding='utf8' , engine='python')
dataRawC06 = pd.read_csv( pathC06  , encoding='utf8' , engine='python')
dataRawC07 = pd.read_csv( pathC07  , encoding='utf8' , engine='python')
dataRawC08 = pd.read_csv( pathC08  , encoding='utf8' , engine='python')
dataRawC09 = pd.read_csv( pathC09  , encoding='utf8' , engine='python')
dataRawC10 = pd.read_csv( pathC10  , encoding='utf8' , engine='python')
dataRawC11 = pd.read_csv( pathC11  , encoding='utf8' , engine='python')
dataRawC12 = pd.read_csv( pathC12  , encoding='utf8' , engine='python')


# se juntan todos los archivos en un solo dataframe
dataClima = pd.concat([dataRawC01
                   , dataRawC02
                   , dataRawC03
                   , dataRawC04
                   , dataRawC05
                   , dataRawC06
                   , dataRawC07
                   , dataRawC08
                   , dataRawC09
                   , dataRawC10
                   , dataRawC11
                   , dataRawC12 ] , axis=0)


# se agregan dos nuevos campos y se da formato a los atributos de fecha y hora
dataClima['Date'] = pd.to_datetime(  dataClima['Date/Time (LST)'] , format='%Y-%m-%d %H:%M'  ).dt.strftime('%Y-%m-%d') 
dataClima['Hour'] = pd.to_datetime(  dataClima['Date/Time (LST)'] , format='%Y-%m-%d %H:%M'  ).dt.hour
dataClima.head()


,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C),...,Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather,Date,Hour
0,-79.4,43.63,TORONTO CITY CENTRE,6158359,2018-01-01 00:00,2018,1,1,00:00,-16.2,...,NaN,102.13,NaN,NaN,NaN,-22.0,NaN,NaN,2018-01-01,0
1,-79.4,43.63,TORONTO CITY CENTRE,6158359,2018-01-01 01:00,2018,1,1,01:00,-16.8,...,NaN,102.10,NaN,NaN,NaN,-20.0,NaN,NaN,2018-01-01,1
2,-79.4,43.63,TORONTO CITY CENTRE,6158359,2018-01-01 02:00,2018,1,1,02:00,-17.3,...,NaN,102.15,NaN,NaN,NaN,-26.0,NaN,NaN,2018-01-01,2
3,-79.4,43.63,TORONTO CITY CENTRE,6158359,2018-01-01 03:00,2018,1,1,03:00,-17.9,...,NaN,102.13,NaN,NaN,NaN,-24.0,NaN,NaN,2018-01-01,3
4,-79.4,43.63,TORONTO CITY CENTRE,6158359,2018-01-01 04:00,2018,1,1,04:00,-17.2,...,NaN,102.09,NaN,NaN,NaN,-21.0,NaN,NaN,2018-01-01,4


In [6]:
# Se une la data de Toronto y la del clima de Toronto
dataToronto = dataTorontoGrp.merge(dataClima , on =[ 'Date', 'Hour']  )


#dataToronto  = dataToronto[[ 'Rented Bike Count' , 'Date','Hour', 'Seasons', 'Station Name','Date/Time (LST)','Time (LST)','Temp (°C)','Dew Point Temp (°C)','Rel Hum (%)','Precip. Amount (mm)','Wind Spd (km/h)','Visibility (km)','Stn Press (kPa)' ]]
dataToronto  = dataToronto[[ 'Rented Bike Count' , 'Date','Hour', 'Seasons', 'Station Name','Date/Time (LST)','Time (LST)','Temp (°C)','Dew Point Temp (°C)','Rel Hum (%)','Precip. Amount (mm)','Wind Spd (km/h)','Visibility (km)' ]]



#Homologacion de nombres, para que quede igual a los de Seul
dataToronto.rename(columns = {'Temp (°C)':'Temperature(°C)' ,  'Rel Hum (%)':'Humidity(%)' ,  'Wind Spd (km/h)':'Wind speed (m/s)' , 'Precip. Amount (mm)':'Rainfall(mm)' , 'Visibility (km)':'Visibility (10m)' ,  'Dew Point Temp (°C)': 'Dew point temperature(°C)'  }, inplace = True)

dataToronto['Wind speed (m/s)'] = round( dataToronto['Wind speed (m/s)'] / 3.6 , 2) 

try:
  dataToronto['Visibility (10m)'] = round( dataToronto['Visibility (10m)'] * 100 , 2 )
except:
  pass
 

# se iguala el periodo con la data que tuiene Seul
dataToronto = dataToronto[  (dataToronto['Date'] >= '2018-01-01') & (dataToronto['Date'] < '2018-12-01')  ]


dataToronto.head()


,Rented Bike Count,Date,Hour,Seasons,Station Name,Date/Time (LST),Time (LST),Temperature(°C),Dew point temperature(°C),Humidity(%),Rainfall(mm),Wind speed (m/s),Visibility (10m)
0,4,2018-01-01,0,Winter,TORONTO CITY CENTRE,2018-01-01 00:00,00:00,-16.2,-20.5,70.0,0.0,2.22,1610.0
1,10,2018-01-01,1,Winter,TORONTO CITY CENTRE,2018-01-01 01:00,01:00,-16.8,-21.1,70.0,0.0,1.11,1610.0
2,6,2018-01-01,2,Winter,TORONTO CITY CENTRE,2018-01-01 02:00,02:00,-17.3,-20.8,75.0,0.0,4.17,1610.0
3,14,2018-01-01,3,Winter,TORONTO CITY CENTRE,2018-01-01 03:00,03:00,-17.9,-21.6,73.0,0.0,2.50,1610.0
4,6,2018-01-01,4,Winter,TORONTO CITY CENTRE,2018-01-01 04:00,04:00,-17.2,-21.1,72.0,0.0,1.39,1610.0


In [7]:
dataSeul = dataSeul.dropna()
dataToronto = dataToronto.dropna()

**Revision de la data limpiada de Seul y Toronto**

Se visualiza los tipos de datos de ambos dataset

In [8]:
dataSeul.dtypes

Date                          object
Rented Bike Count              int64
Hour                           int64
Temperature(°C)              float64
Humidity(%)                    int64
Wind speed (m/s)             float64
Visibility (10m)               int64
Dew point temperature(°C)    float64
Solar Radiation (MJ/m2)      float64
Rainfall(mm)                 float64
Snowfall (cm)                float64
Seasons                       object
Holiday                       object
Functioning Day               object
anho                           int64
dtype: object

In [9]:
dataToronto.dtypes

Rented Bike Count              int64
Date                          object
Hour                           int64
Seasons                       object
Station Name                  object
Date/Time (LST)               object
Time (LST)                    object
Temperature(°C)              float64
Dew point temperature(°C)    float64
Humidity(%)                  float64
Rainfall(mm)                 float64
Wind speed (m/s)             float64
Visibility (10m)             float64
dtype: object

Descripción de caracteristicas

In [10]:
dataSeul.describe().T

,count,mean,std,min,25%,50%,75%,max
Rented Bike Count,8016.0,746.879242,656.708588,0.0,206.00,586.00,1117.00,3556.00
Hour,8016.0,11.500000,6.922618,0.0,5.75,11.50,17.25,23.00
Temperature(°C),8016.0,14.257660,11.475257,-17.8,5.90,15.60,23.20,39.40
Humidity(%),8016.0,58.449102,20.310058,0.0,43.00,58.00,74.00,98.00
Wind speed (m/s),8016.0,1.723715,1.027131,0.0,1.00,1.50,2.30,7.40
Visibility (10m),8016.0,1446.532560,597.333134,27.0,963.00,1699.00,2000.00,2000.00
Dew point temperature(°C),8016.0,5.412038,12.639773,-30.6,-2.40,6.90,15.60,27.20
Solar Radiation (MJ/m2),8016.0,0.603009,0.894237,0.0,0.00,0.02,1.02,3.52
Rainfall(mm),8016.0,0.158146,1.170224,0.0,0.00,0.00,0.00,35.00
Snowfall (cm),8016.0,0.047605,0.361302,0.0,0.00,0.00,0.00,8.80


In [11]:
dataToronto.describe().T

,count,mean,std,min,25%,50%,75%,max
Rented Bike Count,7974.0,230.547529,258.134545,1.0,35.0,134.00,348.00,1602.00
Hour,7974.0,11.539127,6.916075,0.0,6.0,12.00,18.00,23.00
Temperature(°C),7974.0,10.012829,10.438245,-21.9,1.7,9.50,19.80,31.00
Dew point temperature(°C),7974.0,4.991372,11.011835,-27.8,-3.2,4.50,14.70,24.40
Humidity(%),7974.0,72.447078,15.672408,17.0,62.0,73.00,85.00,100.00
Rainfall(mm),7974.0,0.103436,0.886721,0.0,0.0,0.00,0.00,52.30
Wind speed (m/s),7974.0,4.686331,2.797433,0.0,2.5,4.17,6.11,21.11
Visibility (10m),7974.0,1503.816152,316.824908,0.0,1610.0,1610.00,1610.00,1610.00


In [12]:
dataSeul.isna().sum()

Date                         0
Rented Bike Count            0
Hour                         0
Temperature(°C)              0
Humidity(%)                  0
Wind speed (m/s)             0
Visibility (10m)             0
Dew point temperature(°C)    0
Solar Radiation (MJ/m2)      0
Rainfall(mm)                 0
Snowfall (cm)                0
Seasons                      0
Holiday                      0
Functioning Day              0
anho                         0
dtype: int64

In [13]:
dataToronto.isna().sum()

Rented Bike Count            0
Date                         0
Hour                         0
Seasons                      0
Station Name                 0
Date/Time (LST)              0
Time (LST)                   0
Temperature(°C)              0
Dew point temperature(°C)    0
Humidity(%)                  0
Rainfall(mm)                 0
Wind speed (m/s)             0
Visibility (10m)             0
dtype: int64

se observa en ambos dataset que la temperatura y temperatura Dew estan fuertemente relacionadas, por lo que dejaremos fuera la variable temperatura *Dew*

In [14]:
#dataSeul=dataSeul.drop(['Dew point temperature(°C)'],axis=1)
#dataToronto=dataToronto.drop(['Dew Point Temp (°C)'],axis=1)

In [15]:
#df1Corr=pd.DataFrame(dataSeul.corr().unstack().sort_values(ascending=False)['Rented Bike Count'],columns=['Correlation to the target'])
#df1Corr.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

In [16]:
#df1Corr=pd.DataFrame(dataToronto.corr().unstack().sort_values(ascending=False)['Rented Bike Count'],columns=['Correlation to the target'])
#df1Corr.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

Se incorpora infirmacion adicional a la data de mes y tipo de dia, esto para poder distinguir usus de bicicletas  en dia habil que inhabil

In [17]:
dataSeul['Date']=pd.to_datetime(dataRawSeul['Date'],format="%d/%m/%Y")

dataSeul['label_day_night']=dataSeul['Hour'].apply(lambda x : 'Night' if (x >20 or x<5) else( 'Day'))
dataSeul['WeekDay']=dataSeul["Date"].dt.day_name()
dataSeul['Month']=dataSeul['Date'].dt.month

In [18]:
dataToronto['Date']=pd.to_datetime(dataToronto['Date'],format="%Y-%m-%d")

dataToronto['label_day_night']=dataToronto['Hour'].apply(lambda x : 'Night' if (x >20 or x<5) else( 'Day'))
dataToronto['WeekDay']=dataToronto["Date"].dt.day_name()
dataToronto['Month']=dataToronto['Date'].dt.month

In [19]:
#dataSeul.groupby('Hour').sum()['Rented Bike Count'].plot.bar()
#dataSeul.groupby('Hour').sum()['Rented Bike Count'].plot()

In [20]:
#dataToronto.groupby('Hour').sum()['Rented Bike Count'].plot.bar()
#dataToronto.groupby('Hour').sum()['Rented Bike Count'].plot()

**Se genera los archivos para el analisis**

In [21]:


dataSeulFile    =    dataSeul[['Rented Bike Count',	'Date',	'Hour' ,	'Seasons' ,	'Temperature(°C)' ,	'Humidity(%)' ,	'Rainfall(mm)' ,	'label_day_night' , 'WeekDay' ,	'Month' ,	'Wind speed (m/s)' ,	'Visibility (10m)' , 'Dew point temperature(°C)' ]]
dataTorontoFile = dataToronto[['Rented Bike Count',	'Date',	'Hour' ,	'Seasons' ,	'Temperature(°C)' ,	'Humidity(%)' ,	'Rainfall(mm)' ,	'label_day_night' , 'WeekDay' ,	'Month' ,	'Wind speed (m/s)' ,	'Visibility (10m)' , 'Dew point temperature(°C)'  ]]

#dataSeulFile.to_csv(pathBase + '/data/processed/Seul_2018.csv'  ,sep=',' , index = False) 
#dataTorontoFile.to_csv( pathBase + '/data/processed/Toronto_2018.csv'  ,sep=',' , index = False ) 

dataSeulFile['Seul'] = 1
dataTorontoFile['Seul'] = 0

dataBikes = pd.concat([ dataSeulFile , dataTorontoFile ] , axis=0)


dataBikes.to_csv( pathBase + '/data/processed/Bikes_2018.csv'  ,sep=',' , index = False ) 


/var/folders/4m/7h33y3q52x508gscm5tn54xc0000gn/T/ipykernel_82243/2975088298.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSeulFile['Seul'] = 1
/var/folders/4m/7h33y3q52x508gscm5tn54xc0000gn/T/ipykernel_82243/2975088298.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTorontoFile['Seul'] = 0
